In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stat
pd.set_option('display.max_columns', None)
%matplotlib inline

### Feature Processing
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from dateutil import relativedelta

### Feature Selection Techniques
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import VarianceThreshold

### Train Test Split
from sklearn.model_selection import train_test_split

### Model Building
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import  XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

### Metrics
import sklearn.metrics as metrics

### Hyper Paramter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

### Loading and Saving Model
import pickle

### Ensemble Technique - Combining Multiple models
## We call the algorithms like this way

## Then we use Voting Classifier for ensemble learning:
from sklearn.ensemble import VotingClassifier

from keras import backend as K

import re

In [ ]:
train_combined_median = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/train_bur_demo_combined_with_hist_median.xlsx')
train_combined_median.head()

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DPD_Hist_Median_Median,CUR BAL-Hist_Median_Median,AMT OVERDUE - HIST_Median_Median,AMT PAID - HIST_Median_Median
0,0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,275000.0,0.0,0.0,0.0,0.0,0.0,9,True,5,[False True],PRIMARY,9,['PRIMARY'],4,Auto Loan (Personal),5,['Overdraft' 'Auto Loan (Personal)' 'Tractor L...,2,NBF,5,['NAB' 'NBF'],1,Individual,9,['Individual'],3,Closed,5,['Delinquent' 'Active' 'Closed'],0.0,125115.50,615.0,0.0
1,1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,700000.0,0.0,52421.0,0.0,0.0,35.0,13,False,12,[False True],PRIMARY,13,['PRIMARY'],7,Commercial Vehicle Loan,4,['Overdraft' 'Housing Loan' 'Business Loan Gen...,2,NBF,7,['PRB' 'NBF'],2,Individual,9,['Individual' 'Guarantor'],2,Active,7,['Closed' 'Active'],0.0,334547.50,0.0,0.0
2,2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,44000.0,0.0,0.0,0.0,0.0,0.0,31,False,29,[ True False],PRIMARY,31,['PRIMARY'],8,Gold Loan,8,['Two-Wheeler Loan' 'Credit Card' 'Personal Lo...,5,PRB,13,['NBF' 'PRB' 'CCC' 'COP' 'NAB'],3,Individual,26,['Individual' 'Joint' 'Guarantor'],3,Closed,19,['Closed' 'Active' 'Delinquent'],0.0,30303.00,0.0,0.0
3,3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,315000.0,0.0,0.0,0.0,0.0,5.0,4,True,2,[ True False],PRIMARY,4,['PRIMARY'],2,Tractor Loan,3,['Tractor Loan' 'Personal Loan'],1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,['Closed' 'Active'],0.0,195207.75,0.0,0.0
4,4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,200000.0,0.0,6989.0,0.0,0.0,42.0,7,False,6,[False True],PRIMARY,7,['PRIMARY'],3,Tractor Loan,4,['Commercial Vehicle Loan' 'Personal Loan' 'Tr...,3,NBF,4,['NBF' 'NAB' 'PRB'],3,Individual,3,['Guarantor' 'Individual' 'Joint'],4,Active,3,['SUIT FILED (WILFUL DEFAULT)' 'Active' 'Delin...,0.0,172109.00,0.0,0.0


In [ ]:
### preprocessing of Self Indicator
def preproc_self_ind(cols):
  t = cols[0]
  rep = {"[": "","]":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

In [ ]:
train_combined_median['SELF-INDICATOR_unique_values'] = train_combined_median[['SELF-INDICATOR_unique_values']].apply(preproc_self_ind,axis=1)
train_combined_median['SELF-INDICATOR_unique_values'].value_counts()

False  True    51085
True           51010
True False     24441
False           2118
Name: SELF-INDICATOR_unique_values, dtype: int64

In [ ]:
## Convert False True to True False
def preproc_self_ind_false_true(cols):
  t = cols[0]
  rep = {"False  True": "True False"} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

In [ ]:
train_combined_median['SELF-INDICATOR_unique_values'] = train_combined_median[['SELF-INDICATOR_unique_values']].apply(preproc_self_ind_false_true,axis=1)
train_combined_median['SELF-INDICATOR_unique_values'].value_counts()

True False    75526
True          51010
False          2118
Name: SELF-INDICATOR_unique_values, dtype: int64

### Match Type Unique Values

In [ ]:
train_combined_median['MATCH-TYPE_unique_values'].value_counts()

['PRIMARY']                128576
['PRIMARY' 'SECONDARY']        47
['SECONDARY']                  31
Name: MATCH-TYPE_unique_values, dtype: int64

In [ ]:
def preproc_self_match_type(cols):
  t = cols[0]
  rep = {"'": "","[":"","]":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

In [ ]:
train_combined_median['MATCH-TYPE_unique_values'] = train_combined_median[['MATCH-TYPE_unique_values']].apply(preproc_self_match_type,axis=1)
train_combined_median['MATCH-TYPE_unique_values'].value_counts()

PRIMARY              128576
PRIMARY SECONDARY        47
SECONDARY                31
Name: MATCH-TYPE_unique_values, dtype: int64

#### ACCT TYPE Unique Values

In [ ]:
h = train_combined_median.iloc[4]['ACCOUNT-STATUS_unique_values']
h

"['SUIT FILED (WILFUL DEFAULT)' 'Active' 'Delinquent' 'Closed']"

In [ ]:
rep = {"[":"","]":"","\n":""} # define desired replacements here
rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
text = pattern.sub(lambda m: rep[re.escape(m.group(0))], h).strip()
text

"'SUIT FILED (WILFUL DEFAULT)' 'Active' 'Delinquent' 'Closed'"

In [ ]:
text1 = text.replace("' '",'|')
text1

"'SUIT FILED (WILFUL DEFAULT)|Active|Delinquent|Closed'"

In [ ]:
text2 = text1.replace("'","")
text2

'SUIT FILED (WILFUL DEFAULT)|Active|Delinquent|Closed'

In [ ]:
splitt = text2.split('|')
splitt

['SUIT FILED (WILFUL DEFAULT)', 'Active', 'Delinquent', 'Closed']

In [ ]:
final = sorted(splitt)
final

['Active', 'Closed', 'Delinquent', 'SUIT FILED (WILFUL DEFAULT)']

In [ ]:
str(final)

"['Active', 'Closed', 'Delinquent', 'SUIT FILED (WILFUL DEFAULT)']"

##### Function for sorting splitting and String manipulation for all categorical variables 

In [ ]:
def preproc_acc_type_unique(cols):
  t = cols[0]
  rep = {"[":"","]":"","\n":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  text1 = text.replace("' '",'|')
  text2 = text1.replace("'","")
  splitt = text2.split('|')
  final = sorted(splitt)
  final_text = str(final)
  return final_text

In [ ]:
train_combined_median['ACCT-TYPE_unique_values'] = train_combined_median[['ACCT-TYPE_unique_values']].apply(preproc_acc_type_unique,axis=1)

In [ ]:
len(train_combined_median['ACCT-TYPE_unique_values'].unique())

7782

These Function can be applied to all other cat unique values

In [ ]:
len(train_combined_median['CONTRIBUTOR-TYPE_unique_values'].unique())

1046

In [ ]:
train_combined_median['CONTRIBUTOR-TYPE_unique_values'] = train_combined_median[['CONTRIBUTOR-TYPE_unique_values']].apply(preproc_acc_type_unique,axis=1)

In [ ]:
len(train_combined_median['CONTRIBUTOR-TYPE_unique_values'].unique())

182

OWNERSHIP-IND_unique_values

In [ ]:
len(train_combined_median['OWNERSHIP-IND_unique_values'].unique())

75

In [ ]:
train_combined_median['OWNERSHIP-IND_unique_values'] = train_combined_median[['OWNERSHIP-IND_unique_values']].apply(preproc_acc_type_unique,axis=1)

In [ ]:
len(train_combined_median['OWNERSHIP-IND_unique_values'].unique())

18

ACCOUNT-STATUS_unique_values

In [ ]:
len(train_combined_median['ACCOUNT-STATUS_unique_values'].unique())

390

In [ ]:
train_combined_median['ACCOUNT-STATUS_unique_values'] = train_combined_median[['ACCOUNT-STATUS_unique_values']].apply(preproc_acc_type_unique,axis=1)

In [ ]:
len(train_combined_median['ACCOUNT-STATUS_unique_values'].unique())

91

In [ ]:
train_combined_median.head(10)

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DPD_Hist_Median_Median,CUR BAL-Hist_Median_Median,AMT OVERDUE - HIST_Median_Median,AMT PAID - HIST_Median_Median
0,0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,275000.0,0.0,0.0,0.0,0.0,0.0,9,True,5,True False,PRIMARY,9,PRIMARY,4,Auto Loan (Personal),5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,"['NAB', 'NBF']",1,Individual,9,['Individual'],3,Closed,5,"['Active', 'Closed', 'Delinquent']",0.0,125115.50,615.0,0.0
1,1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,700000.0,0.0,52421.0,0.0,0.0,35.0,13,False,12,True False,PRIMARY,13,PRIMARY,7,Commercial Vehicle Loan,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,"['NBF', 'PRB']",2,Individual,9,"['Guarantor', 'Individual']",2,Active,7,"['Active', 'Closed']",0.0,334547.50,0.0,0.0
2,2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,44000.0,0.0,0.0,0.0,0.0,0.0,31,False,29,True False,PRIMARY,31,PRIMARY,8,Gold Loan,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,"['CCC', 'COP', 'NAB', 'NBF', 'PRB']",3,Individual,26,"['Guarantor', 'Individual', 'Joint']",3,Closed,19,"['Active', 'Closed', 'Delinquent']",0.0,30303.00,0.0,0.0
3,3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,315000.0,0.0,0.0,0.0,0.0,5.0,4,True,2,True False,PRIMARY,4,PRIMARY,2,Tractor Loan,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,"['Active', 'Closed']",0.0,195207.75,0.0,0.0
4,4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,200000.0,0.0,6989.0,0.0,0.0,42.0,7,False,6,True False,PRIMARY,7,PRIMARY,3,Tractor Loan,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,"['NAB', 'NBF', 'PRB']",3,Individual,3,"['Guarantor', 'Individual', 'Joint']",4,Active,3,"['Active', 'Closed', 'Delinquent', 'SUIT FILED...",0.0,172109.00,0.0,0.0
5,5,0.168888,5,48,400000,280000.0,53000.0,1019,54689,40,42083.33,0.269824,493885,0,0,1,0,1,1,1,9,280000.0,0.0,0.0,0.0,0.0,0.0,1,True,1,True,PRIMARY,1,PRIMARY,1,Tractor Loan,1,['Tractor Loan'],1,NBF,1,['NBF'],1,Individual,1,['Individual'],1,Closed,1,['Closed'],0.0,141074.50,0.0,0.0
6,6,0.186877,5,48,716000,450000.0,2000.0,1019,54689,23,46221.00,0.269824,493889,0,0,1,0,1,1,1,10,395253.0,0.0,0.0,0.0,0.0,46.0,3,False,2,True False,PRIMARY,3,PRIMARY,2,Tractor Loan,2,"['Commercial Vehicle Loan', 'Tractor Loan']",1,NBF,3,['NBF'],2,Individual,2,"['Guarantor', 'Individual']",2,Closed,2,"['Closed', 'Delinquent']",0.0,222506.00,0.0,0.0
7,7,0.186877,5,48,600000,360000.0,11000.0,1187,21232,41,46195.08,0.269824,493114,0,0,1,0,1,1,1,11,350000.0,0.0,0.0,0.0,0.0,0.0,25,False,24,True False,PRIMARY,25,PRIMARY,4,Tractor Loan,17,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,21,"['NBF', 'PRB']",3,Individual,12,"['Guarantor', 'Individual', 'Joint']",2,Closed,17,"['Active', 'Closed']",0.0,99635.50

In [ ]:
train_combined_median = train_combined_median.iloc[:,1:]
train_combined_median

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DPD_Hist_Median_Median,CUR BAL-Hist_Median_Median,AMT OVERDUE - HIST_Median_Median,AMT PAID - HIST_Median_Median
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,275000.0,0.0,0.0,0.0,0.0,0.0,9,True,5,True False,PRIMARY,9,PRIMARY,4,Auto Loan (Personal),5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,"['NAB', 'NBF']",1,Individual,9,['Individual'],3,Closed,5,"['Active', 'Closed', 'Delinquent']",0.0,125115.50,615.0,0.0
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,700000.0,0.0,52421.0,0.0,0.0,35.0,13,False,12,True False,PRIMARY,13,PRIMARY,7,Commercial Vehicle Loan,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,"['NBF', 'PRB']",2,Individual,9,"['Guarantor', 'Individual']",2,Active,7,"['Active', 'Closed']",0.0,334547.50,0.0,0.0
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,44000.0,0.0,0.0,0.0,0.0,0.0,31,False,29,True False,PRIMARY,31,PRIMARY,8,Gold Loan,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,"['CCC', 'COP', 'NAB', 'NBF', 'PRB']",3,Individual,26,"['Guarantor', 'Individual', 'Joint']",3,Closed,19,"['Active', 'Closed', 'Delinquent']",0.0,30303.00,0.0,0.0
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,315000.0,0.0,0.0,0.0,0.0,5.0,4,True,2,True False,PRIMARY,4,PRIMARY,2,Tractor Loan,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,"['Active', 'Closed']",0.0,195207.75,0.0,0.0
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,200000.0,0.0,6989.0,0.0,0.0,42.0,7,False,6,True False,PRIMARY,7,PRIMARY,3,Tractor Loan,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,"['NAB', 'NBF', 'PRB']",3,Individual,3,"['Guarantor', 'Individual', 'Joint']",4,Active,3,"['Active', 'Closed', 'Delinquent', 'SUIT FILED...",0.0,172109.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128649,0.668451,424,24,470000,265601.0,76800.0,1568,48879,25,65333.33,0.533554,131403,3,1,0,0,1,1,1,143390,1043800.5,0.0,933815.0,0.0,0.0,60.5,2,True,1,True False,PRIMARY,2,PRIMARY,2,Property Loan,1,"['Property Loan', 'Tractor Loan']",2,NBF,1,"['NBF', 'PRB']",1,Individual,2,['Individual'],1,Active,2,['Active'],0.0,1003495.50,0.0,0.0
128650,0.668451,424,24,460000,275630.0,80100.0,1568,48879,25,83333.33,0.533554,131403,0,1,0,0,1,1,1,143391,275630.0,0.0,73890.0,0.0,0.0,0.0,1,True,1,True,PRIMARY,1,PRIMARY,1,Tractor Loan,1,['Tractor Loan'],1,NBF,1,['NBF'],1,Individual,1,['Individual'],1,Active,1,['Active'],0.0,204926.00,0.0,0.0
128651,0.668451,424,23,545000,300733.0,15277.0,1568,44118,36,248500.00,0.533554,131024,0,0,1,0,1,0,1,143393,140366.5,0.0,5843.5,0.0,0.0,18.0,4,False,3,True False,PRIMARY,4,P

In [ ]:
train_combined_median.to_excel('train_bur_demo_median_updated_string_manipulation.xlsx')

#### Cardinality of all categorical Variables

In [ ]:
train_combined_median.dtypes

PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
top_up_class                                  int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M                                         int64
ID          

In [ ]:
train_combined_median['SELF-INDICATOR_Top_value'] = train_combined_median['SELF-INDICATOR_Top_value'].astype('object')

In [ ]:
train_combined_median['SELF-INDICATOR_Top_value'].head()

0     True
1    False
2    False
3     True
4    False
Name: SELF-INDICATOR_Top_value, dtype: object

In [ ]:
train_combined_median.dtypes

PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
top_up_class                                  int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M                                         int64
ID          

-----------------------------

##### Creating all new Features for TEST Data

In [ ]:
test_data_bur = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/test_data_bur.xlsx')
test_data_bur.head()

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median
0,0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0
1,1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0
2,2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5
3,3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0
4,4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0


In [ ]:
test_data_bur = test_data_bur.iloc[:,1:]
test_data_bur.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0


In [ ]:
bur_test = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/bureau_data_test_manipulated.xlsx')
bur_test.head()

,Unnamed: 0,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,0,4,False,PRIMARY,Credit Card,CCC,2019-11-17,Individual,Active,2019-09-26,NaN,NaT,"10,000",588,0,589,NaN,0,0,NaN,"20191117,",000,"589,","0,","0,",0
1,1,4,False,PRIMARY,Housing Loan,HFC,2019-10-31,Joint,Active,2017-06-21,NaN,2019-10-15,NaN,2500000,30797,2107227,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"2107227,2122196,2136877,2151448,2165910,218026...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120
2,2,4,False,PRIMARY,Property Loan,HFC,2019-10-31,Joint,Active,2017-06-27,NaN,2019-10-15,NaN,99000,1223,83458,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"83458,84051,84632,85209,85781,86349,86913,8747...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120
3,3,4,True,PRIMARY,Tractor Loan,NBF,2015-10-31,Individual,Closed,2011-12-29,2015-10-13 00:00:00,NaT,NaN,365000,0,0,NaN,0,0,NaN,"20151031,20150901,20150831,20150731,20150630,2...",000DDD0270000000000000000000000000000000000000...,"0,,21548,32097,42500,52758,62874,72850,82688,9...","0,,11000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",0
4,4,5,False,PRIMARY,Gold Loan,PRB,2017-08-31,Individual,Closed,2016-05-04,2017-06-15 00:00:00,2017-06-15,NaN,25801,0,0,NaN,0,0,NaN,"20170630,20170501,20170401,20170331,20170201,2...",000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,25846,,,25846,,,25846,,,25846,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",0


In [ ]:
bur_test = bur_test.iloc[:,1:]
bur_test.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,4,False,PRIMARY,Credit Card,CCC,2019-11-17,Individual,Active,2019-09-26,NaN,NaT,"10,000",588,0,589,NaN,0,0,NaN,"20191117,",000,"589,","0,","0,",0
1,4,False,PRIMARY,Housing Loan,HFC,2019-10-31,Joint,Active,2017-06-21,NaN,2019-10-15,NaN,2500000,30797,2107227,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"2107227,2122196,2136877,2151448,2165910,218026...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120
2,4,False,PRIMARY,Property Loan,HFC,2019-10-31,Joint,Active,2017-06-27,NaN,2019-10-15,NaN,99000,1223,83458,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"83458,84051,84632,85209,85781,86349,86913,8747...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120
3,4,True,PRIMARY,Tractor Loan,NBF,2015-10-31,Individual,Closed,2011-12-29,2015-10-13 00:00:00,NaT,NaN,365000,0,0,NaN,0,0,NaN,"20151031,20150901,20150831,20150731,20150630,2...",000DDD0270000000000000000000000000000000000000...,"0,,21548,32097,42500,52758,62874,72850,82688,9...","0,,11000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",0
4,5,False,PRIMARY,Gold Loan,PRB,2017-08-31,Individual,Closed,2016-05-04,2017-06-15 00:00:00,2017-06-15,NaN,25801,0,0,NaN,0,0,NaN,"20170630,20170501,20170401,20170331,20170201,2...",000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,25846,,,25846,,,25846,,,25846,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",0


In [ ]:
bur_data_test_groupby = bur_test.groupby('ID')
bur_data_test_groupby

In [ ]:
test_data_bur.shape

(14745, 25)

In [ ]:
test_data_bur.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0


In [ ]:
def extract_categorical_features_values_bur(bur_data_test_groupby,features,test_data_bur):
  for feature in features:
    dd = bur_data_test_groupby[feature].describe().reset_index()
    ddd= bur_data_test_groupby[feature].unique().to_frame().reset_index()
    dddd = pd.merge(dd,ddd,on='ID')
    dddd.columns = ['ID','{}_Bureau_Traits_Total_Count'.format(feature),'{}_Unique_count'.format(feature),'{}_Top_value'.format(feature),
                  '{}_Top_value_freq'.format(feature),'{}_unique_values'.format(feature)]
    test_data_bur = pd.merge(test_data_bur,dddd,on='ID')
  return test_data_bur
    
test_data_bur_cat_feat = extract_categorical_features_values_bur(bur_data_test_groupby,['SELF-INDICATOR','MATCH-TYPE','ACCT-TYPE','CONTRIBUTOR-TYPE','OWNERSHIP-IND','ACCOUNT-STATUS'],test_data_bur)

In [ ]:
test_data_bur_cat_feat.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Unique_count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Bureau_Traits_Total_Count,MATCH-TYPE_Unique_count,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Bureau_Traits_Total_Count,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Bureau_Traits_Total_Count,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Bureau_Traits_Total_Count,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Bureau_Traits_Total_Count,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0,4,2,False,3,"[False, True]",4,1,PRIMARY,4,[PRIMARY],4,4,Tractor Loan,1,"[Credit Card, Housing Loan, Property Loan, Tra...",4,3,HFC,2,"[CCC, HFC, NBF]",4,2,Individual,2,"[Individual, Joint]",4,2,Active,3,"[Active, Closed]"
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0,8,2,False,7,"[False, True]",8,1,PRIMARY,8,[PRIMARY],8,5,Gold Loan,3,"[Gold Loan, Consumer Loan, Personal Loan, Trac...",8,4,NBF,3,"[PRB, NBF, HFC, MFI]",8,2,Individual,6,"[Individual, Joint]",8,2,Closed,6,"[Closed, Active]"
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5,12,2,False,10,"[False, True]",12,1,PRIMARY,12,[PRIMARY],12,4,Auto Loan (Personal),7,"[Auto Loan (Personal), Tractor Loan, Gold Loan...",12,2,NBF,11,"[NBF, PRB]",12,3,Individual,6,"[Guarantor, Individual, Joint]",12,2,Closed,11,"[Closed, Active]"
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0,2,2,True,1,"[False, True]",2,1,PRIMARY,2,[PRIMARY],2,2,Business Loan Priority Sector Agriculture,1,"[Business Loan Priority Sector Agriculture, T...",2,2,NBF,1,"[NAB, NBF]",2,1,Individual,2,[Individual],2,2,Active,1,"[Active, Closed]"
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0,2,2,True,1,"[False, True]",2,1,PRIMARY,2,[PRIMARY],2,2,Auto Loan (Personal),1,"[Auto Loan (Personal), Tractor Loan]",2,1,NBF,2,[NBF],2,2,Guarantor,1,"[Guarantor, Individual]",2,2,Active,1,"[Active, Closed]"


In [ ]:
test_data_bur_cat_feat.shape

(14745, 55)

In [ ]:
test_data_bur_cat_feat.columns

Index(['PaymentMode', 'BranchID', 'Tenure', 'AssetCost', 'AmountFinance',
       'EMI', 'ManufacturerID', 'SupplierID', 'AGE', 'MonthlyIncome', 'State',
       'ZiPCODE', 'Frequency_Half Yearly', 'Frequency_Monthly',
       'Frequency_Quatrly', 'InstlmentMode_Arrear', 'LoanStatus_Closed',
       'SEX_M', 'ID', 'DISBURSED-AMT/HIGH CREDIT_Median',
       'INSTALLMENT-AMT_Median', 'CURRENT-BAL_Median', 'OVERDUE-AMT_Median',
       'WRITE-OFF-AMT_Median', 'TENURE_Median',
       'SELF-INDICATOR_Bureau_Traits_Total_Count',
       'SELF-INDICATOR_Unique_count', 'SELF-INDICATOR_Top_value',
       'SELF-INDICATOR_Top_value_freq', 'SELF-INDICATOR_unique_values',
       'MATCH-TYPE_Bureau_Traits_Total_Count', 'MATCH-TYPE_Unique_count',
       'MATCH-TYPE_Top_value', 'MATCH-TYPE_Top_value_freq',
       'MATCH-TYPE_unique_values', 'ACCT-TYPE_Bureau_Traits_Total_Count',
       'ACCT-TYPE_Unique_count', 'ACCT-TYPE_Top_value',
       'ACCT-TYPE_Top_value_freq', 'ACCT-TYPE_unique_values',
       'CONT

In [ ]:
test_data_bur_cat_feat = test_data_bur_cat_feat.drop(['SELF-INDICATOR_Unique_count','MATCH-TYPE_Unique_count','MATCH-TYPE_Bureau_Traits_Total_Count','ACCT-TYPE_Bureau_Traits_Total_Count','CONTRIBUTOR-TYPE_Bureau_Traits_Total_Count','OWNERSHIP-IND_Bureau_Traits_Total_Count','ACCOUNT-STATUS_Bureau_Traits_Total_Count'],axis=1)
test_data_bur_cat_feat.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0,4,False,3,"[False, True]",PRIMARY,4,[PRIMARY],4,Tractor Loan,1,"[Credit Card, Housing Loan, Property Loan, Tra...",3,HFC,2,"[CCC, HFC, NBF]",2,Individual,2,"[Individual, Joint]",2,Active,3,"[Active, Closed]"
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0,8,False,7,"[False, True]",PRIMARY,8,[PRIMARY],5,Gold Loan,3,"[Gold Loan, Consumer Loan, Personal Loan, Trac...",4,NBF,3,"[PRB, NBF, HFC, MFI]",2,Individual,6,"[Individual, Joint]",2,Closed,6,"[Closed, Active]"
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5,12,False,10,"[False, True]",PRIMARY,12,[PRIMARY],4,Auto Loan (Personal),7,"[Auto Loan (Personal), Tractor Loan, Gold Loan...",2,NBF,11,"[NBF, PRB]",3,Individual,6,"[Guarantor, Individual, Joint]",2,Closed,11,"[Closed, Active]"
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Business Loan Priority Sector Agriculture,1,"[Business Loan Priority Sector Agriculture, T...",2,NBF,1,"[NAB, NBF]",1,Individual,2,[Individual],2,Active,1,"[Active, Closed]"
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Auto Loan (Personal),1,"[Auto Loan (Personal), Tractor Loan]",1,NBF,2,[NBF],2,Guarantor,1,"[Guarantor, Individual]",2,Active,1,"[Active, Closed]"


In [ ]:
test_data_bur_cat_feat.shape

(14745, 48)

In [ ]:
test_data_bur_cat_feat.to_excel('test_data_bur_cat_included.xlsx')

#### Remaining HIST Features 

In [ ]:
bur_test.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE
0,4,False,PRIMARY,Credit Card,CCC,2019-11-17,Individual,Active,2019-09-26,NaN,NaT,"10,000",588,0,589,NaN,0,0,NaN,"20191117,",000,"589,","0,","0,",0
1,4,False,PRIMARY,Housing Loan,HFC,2019-10-31,Joint,Active,2017-06-21,NaN,2019-10-15,NaN,2500000,30797,2107227,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"2107227,2122196,2136877,2151448,2165910,218026...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120
2,4,False,PRIMARY,Property Loan,HFC,2019-10-31,Joint,Active,2017-06-27,NaN,2019-10-15,NaN,99000,1223,83458,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"83458,84051,84632,85209,85781,86349,86913,8747...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120
3,4,True,PRIMARY,Tractor Loan,NBF,2015-10-31,Individual,Closed,2011-12-29,2015-10-13 00:00:00,NaT,NaN,365000,0,0,NaN,0,0,NaN,"20151031,20150901,20150831,20150731,20150630,2...",000DDD0270000000000000000000000000000000000000...,"0,,21548,32097,42500,52758,62874,72850,82688,9...","0,,11000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",0
4,5,False,PRIMARY,Gold Loan,PRB,2017-08-31,Individual,Closed,2016-05-04,2017-06-15 00:00:00,2017-06-15,NaN,25801,0,0,NaN,0,0,NaN,"20170630,20170501,20170401,20170331,20170201,2...",000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,25846,,,25846,,,25846,,,25846,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",0


In [ ]:
bur_test.isnull().sum()

ID                               0
SELF-INDICATOR                   0
MATCH-TYPE                       0
ACCT-TYPE                        0
CONTRIBUTOR-TYPE                 0
DATE-REPORTED                    0
OWNERSHIP-IND                    0
ACCOUNT-STATUS                   0
DISBURSED-DT                    44
CLOSE-DT                     31453
LAST-PAYMENT-DATE            32550
CREDIT-LIMIT/SANC AMT        62421
DISBURSED-AMT/HIGH CREDIT        0
INSTALLMENT-AMT                  0
CURRENT-BAL                      0
INSTALLMENT-FREQUENCY        48243
OVERDUE-AMT                      0
WRITE-OFF-AMT                    0
ASSET_CLASS                  34902
REPORTED DATE - HIST          2561
DPD - HIST                    2565
CUR BAL - HIST                2561
AMT OVERDUE - HIST            2561
AMT PAID - HIST               2672
TENURE                           0
dtype: int64

In [ ]:
bur_test.dtypes

ID                                    int64
SELF-INDICATOR                         bool
MATCH-TYPE                           object
ACCT-TYPE                            object
CONTRIBUTOR-TYPE                     object
DATE-REPORTED                datetime64[ns]
OWNERSHIP-IND                        object
ACCOUNT-STATUS                       object
DISBURSED-DT                 datetime64[ns]
CLOSE-DT                             object
LAST-PAYMENT-DATE            datetime64[ns]
CREDIT-LIMIT/SANC AMT                object
DISBURSED-AMT/HIGH CREDIT             int64
INSTALLMENT-AMT                       int64
CURRENT-BAL                           int64
INSTALLMENT-FREQUENCY                object
OVERDUE-AMT                           int64
WRITE-OFF-AMT                         int64
ASSET_CLASS                          object
REPORTED DATE - HIST                 object
DPD - HIST                           object
CUR BAL - HIST                       object
AMT OVERDUE - HIST              

In [ ]:
bur_test['DPD - HIST'] = bur_test['DPD - HIST'].fillna('0')
bur_test.loc[bur_test['DPD - HIST'].str.contains('E'),'DPD - HIST']='0'

In [ ]:
import statistics
def extract_features_dpd(cols):
  dpdd = cols[0]
  splitted_string= [dpdd[i:i+3] for i in range(0, len(dpdd), 3)]
  resD = [ele.replace('DDD','000') for ele in splitted_string]
  res_final = [ele.replace('XXX','000') for ele in resD]
  res_int = [int(float(i)) for i in res_final]
  dpdmedian = statistics.median(res_int)
  return dpdmedian

In [ ]:
bur_test['DPD_Hist_Median'] = bur_test[['DPD - HIST']].apply(extract_features_dpd,axis=1)
bur_test.head()

Curr Bal Hist

In [ ]:
import statistics
def extract_features_curr_bal_hist(cols):
  crr = cols[0]
  crrlst= crr.split(',')
  test_list = list(filter(None, crrlst))
  if not test_list:
    return 0
  else:
    res_int = [int(i) for i in test_list]
    crrmedian = statistics.median(res_int)
    return crrmedian 

In [ ]:
bur_test['CUR BAL - HIST'] = bur_test['CUR BAL - HIST'].fillna('0')
bur_test['AMT OVERDUE - HIST'] = bur_test['AMT OVERDUE - HIST'].fillna('0')
bur_test['AMT PAID - HIST'] = bur_test['AMT PAID - HIST'].fillna('0')
bur_test.isnull().sum() 

ID                               0
SELF-INDICATOR                   0
MATCH-TYPE                       0
ACCT-TYPE                        0
CONTRIBUTOR-TYPE                 0
DATE-REPORTED                    0
OWNERSHIP-IND                    0
ACCOUNT-STATUS                   0
DISBURSED-DT                    44
CLOSE-DT                     31453
LAST-PAYMENT-DATE            32550
CREDIT-LIMIT/SANC AMT        62421
DISBURSED-AMT/HIGH CREDIT        0
INSTALLMENT-AMT                  0
CURRENT-BAL                      0
INSTALLMENT-FREQUENCY        48243
OVERDUE-AMT                      0
WRITE-OFF-AMT                    0
ASSET_CLASS                  34902
REPORTED DATE - HIST          2561
DPD - HIST                       0
CUR BAL - HIST                   0
AMT OVERDUE - HIST               0
AMT PAID - HIST                  0
TENURE                           0
DPD_Hist_Median                  0
dtype: int64

In [ ]:
bur_test['CUR BAL-Hist_Median'] = bur_test[['CUR BAL - HIST']].apply(extract_features_curr_bal_hist,axis=1)

#### Extract Features Crr function can also be applied to extract median values for all other Feature Columns

In [ ]:
import statistics
def extract_features_hist(cols):
  crr = cols[0]
  crrlst= crr.split(',')
  test_list = list(filter(None, crrlst))
  if not test_list:
    return 0
  else:
    res_int = [int(float(i)) for i in test_list]
    crrmedian = statistics.median(res_int)
    return crrmedian 

In [ ]:
bur_test['AMT OVERDUE - HIST_Median'] = bur_test[['AMT OVERDUE - HIST']].apply(extract_features_hist,axis=1)
bur_test['AMT PAID - HIST_Median'] = bur_test[['AMT PAID - HIST']].apply(extract_features_hist,axis=1)

In [ ]:
bur_test.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,DPD_Hist_Median,CUR BAL-Hist_Median,AMT OVERDUE - HIST_Median,AMT PAID - HIST_Median
0,4,False,PRIMARY,Credit Card,CCC,2019-11-17,Individual,Active,2019-09-26,NaN,NaT,"10,000",588,0,589,NaN,0,0,NaN,"20191117,",000,"589,","0,","0,",0,0.0,589.0,0.0,0.0
1,4,False,PRIMARY,Housing Loan,HFC,2019-10-31,Joint,Active,2017-06-21,NaN,2019-10-15,NaN,2500000,30797,2107227,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"2107227,2122196,2136877,2151448,2165910,218026...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120,0.0,2306032.0,0.0,0.0
2,4,False,PRIMARY,Property Loan,HFC,2019-10-31,Joint,Active,2017-06-27,NaN,2019-10-15,NaN,99000,1223,83458,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"83458,84051,84632,85209,85781,86349,86913,8747...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120,0.0,91325.0,0.0,0.0
3,4,True,PRIMARY,Tractor Loan,NBF,2015-10-31,Individual,Closed,2011-12-29,2015-10-13 00:00:00,NaT,NaN,365000,0,0,NaN,0,0,NaN,"20151031,20150901,20150831,20150731,20150630,2...",000DDD0270000000000000000000000000000000000000...,"0,,21548,32097,42500,52758,62874,72850,82688,9...","0,,11000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",0,0.0,173870.0,0.0,0.0
4,5,False,PRIMARY,Gold Loan,PRB,2017-08-31,Individual,Closed,2016-05-04,2017-06-15 00:00:00,2017-06-15,NaN,25801,0,0,NaN,0,0,NaN,"20170630,20170501,20170401,20170331,20170201,2...",000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,25846,,,25846,,,25846,,,25846,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",0,0.0,25846.0,0.0,0.0


In [ ]:
test_data_bur_cat_feat.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0,4,False,3,"[False, True]",PRIMARY,4,[PRIMARY],4,Tractor Loan,1,"[Credit Card, Housing Loan, Property Loan, Tra...",3,HFC,2,"[CCC, HFC, NBF]",2,Individual,2,"[Individual, Joint]",2,Active,3,"[Active, Closed]"
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0,8,False,7,"[False, True]",PRIMARY,8,[PRIMARY],5,Gold Loan,3,"[Gold Loan, Consumer Loan, Personal Loan, Trac...",4,NBF,3,"[PRB, NBF, HFC, MFI]",2,Individual,6,"[Individual, Joint]",2,Closed,6,"[Closed, Active]"
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5,12,False,10,"[False, True]",PRIMARY,12,[PRIMARY],4,Auto Loan (Personal),7,"[Auto Loan (Personal), Tractor Loan, Gold Loan...",2,NBF,11,"[NBF, PRB]",3,Individual,6,"[Guarantor, Individual, Joint]",2,Closed,11,"[Closed, Active]"
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Business Loan Priority Sector Agriculture,1,"[Business Loan Priority Sector Agriculture, T...",2,NBF,1,"[NAB, NBF]",1,Individual,2,[Individual],2,Active,1,"[Active, Closed]"
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Auto Loan (Personal),1,"[Auto Loan (Personal), Tractor Loan]",1,NBF,2,[NBF],2,Guarantor,1,"[Guarantor, Individual]",2,Active,1,"[Active, Closed]"


In [ ]:
test_data_bur_cat_feat.shape

(14745, 48)

In [ ]:
bur_test.to_excel('test_data_bur_hist_features.xlsx')

In [ ]:
bur_test.columns

Index(['ID', 'SELF-INDICATOR', 'MATCH-TYPE', 'ACCT-TYPE', 'CONTRIBUTOR-TYPE',
       'DATE-REPORTED', 'OWNERSHIP-IND', 'ACCOUNT-STATUS', 'DISBURSED-DT',
       'CLOSE-DT', 'LAST-PAYMENT-DATE', 'CREDIT-LIMIT/SANC AMT',
       'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
       'INSTALLMENT-FREQUENCY', 'OVERDUE-AMT', 'WRITE-OFF-AMT', 'ASSET_CLASS',
       'REPORTED DATE - HIST', 'DPD - HIST', 'CUR BAL - HIST',
       'AMT OVERDUE - HIST', 'AMT PAID - HIST', 'TENURE', 'DPD_Hist_Median',
       'CUR BAL-Hist_Median', 'AMT OVERDUE - HIST_Median',
       'AMT PAID - HIST_Median'],
      dtype='object')

In [ ]:
bur_test_groupby = bur_test.groupby('ID')
bur_test_groupby

In [ ]:
def extract_median_values(bur_test_groupby,features,test_data_bur_cat_feat):
  for feature in features:
    feat_median = bur_test_groupby[feature].median()
    d=feat_median.to_frame().reset_index()
    d.columns = ['ID','{}_Median'.format(feature)]
    test_data_bur_cat_feat = pd.merge(test_data_bur_cat_feat,d,on='ID')
  return test_data_bur_cat_feat

test_data_bur_demo = extract_median_values(bur_test_groupby,['DPD_Hist_Median','CUR BAL-Hist_Median','AMT OVERDUE - HIST_Median','AMT PAID - HIST_Median'],test_data_bur_cat_feat)

In [ ]:
test_data_bur_demo.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DPD_Hist_Median_Median,CUR BAL-Hist_Median_Median,AMT OVERDUE - HIST_Median_Median,AMT PAID - HIST_Median_Median
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0,4,False,3,"[False, True]",PRIMARY,4,[PRIMARY],4,Tractor Loan,1,"[Credit Card, Housing Loan, Property Loan, Tra...",3,HFC,2,"[CCC, HFC, NBF]",2,Individual,2,"[Individual, Joint]",2,Active,3,"[Active, Closed]",0.0,132597.5,0.0,0.0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0,8,False,7,"[False, True]",PRIMARY,8,[PRIMARY],5,Gold Loan,3,"[Gold Loan, Consumer Loan, Personal Loan, Trac...",4,NBF,3,"[PRB, NBF, HFC, MFI]",2,Individual,6,"[Individual, Joint]",2,Closed,6,"[Closed, Active]",0.0,98453.5,0.0,0.0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5,12,False,10,"[False, True]",PRIMARY,12,[PRIMARY],4,Auto Loan (Personal),7,"[Auto Loan (Personal), Tractor Loan, Gold Loan...",2,NBF,11,"[NBF, PRB]",3,Individual,6,"[Guarantor, Individual, Joint]",2,Closed,11,"[Closed, Active]",0.0,22018.0,0.0,0.0
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Business Loan Priority Sector Agriculture,1,"[Business Loan Priority Sector Agriculture, T...",2,NBF,1,"[NAB, NBF]",1,Individual,2,[Individual],2,Active,1,"[Active, Closed]",0.0,139003.0,0.0,0.0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Auto Loan (Personal),1,"[Auto Loan (Personal), Tractor Loan]",1,NBF,2,[NBF],2,Guarantor,1,"[Guarantor, Individual]",2,Active,1,"[Active, Closed]",0.0,254296.0,0.0,265000.0


In [ ]:
test_data_bur_demo.to_excel('test_bur_demo_combined_with_hist_median.xlsx')

In [ ]:
test_data_bur_demo.shape

(14745, 52)

#### Using Mean Values 

In [ ]:
bur_test.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,DPD_Hist_Median,CUR BAL-Hist_Median,AMT OVERDUE - HIST_Median,AMT PAID - HIST_Median
0,4,False,PRIMARY,Credit Card,CCC,2019-11-17,Individual,Active,2019-09-26,NaN,NaT,"10,000",588,0,589,NaN,0,0,NaN,"20191117,",000,"589,","0,","0,",0,0.0,589.0,0.0,0.0
1,4,False,PRIMARY,Housing Loan,HFC,2019-10-31,Joint,Active,2017-06-21,NaN,2019-10-15,NaN,2500000,30797,2107227,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"2107227,2122196,2136877,2151448,2165910,218026...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120,0.0,2306032.0,0.0,0.0
2,4,False,PRIMARY,Property Loan,HFC,2019-10-31,Joint,Active,2017-06-27,NaN,2019-10-15,NaN,99000,1223,83458,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"83458,84051,84632,85209,85781,86349,86913,8747...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120,0.0,91325.0,0.0,0.0
3,4,True,PRIMARY,Tractor Loan,NBF,2015-10-31,Individual,Closed,2011-12-29,2015-10-13 00:00:00,NaT,NaN,365000,0,0,NaN,0,0,NaN,"20151031,20150901,20150831,20150731,20150630,2...",000DDD0270000000000000000000000000000000000000...,"0,,21548,32097,42500,52758,62874,72850,82688,9...","0,,11000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",0,0.0,173870.0,0.0,0.0
4,5,False,PRIMARY,Gold Loan,PRB,2017-08-31,Individual,Closed,2016-05-04,2017-06-15 00:00:00,2017-06-15,NaN,25801,0,0,NaN,0,0,NaN,"20170630,20170501,20170401,20170331,20170201,2...",000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,25846,,,25846,,,25846,,,25846,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",0,0.0,25846.0,0.0,0.0


In [ ]:
import statistics
def extract_features_dpd(cols):
  dpdd = cols[0]
  splitted_string= [dpdd[i:i+3] for i in range(0, len(dpdd), 3)]
  resD = [ele.replace('DDD','000') for ele in splitted_string]
  res_final = [ele.replace('XXX','000') for ele in resD]
  res_int = [int(float(i)) for i in res_final]
  dpdmedian = statistics.mean(res_int)
  return dpdmedian

In [ ]:
bur_test['DPD_Hist_Mean'] = bur_test[['DPD - HIST']].apply(extract_features_dpd,axis=1)

In [ ]:
import statistics
def extract_features_curr_bal_hist(cols):
  crr = cols[0]
  crrlst= crr.split(',')
  test_list = list(filter(None, crrlst))
  if not test_list:
    return 0
  else:
    res_int = [int(i) for i in test_list]
    crrmedian = statistics.mean(res_int)
    return crrmedian 

In [ ]:
bur_test['CUR BAL-Hist_Mean'] = bur_test[['CUR BAL - HIST']].apply(extract_features_curr_bal_hist,axis=1)

In [ ]:
import statistics
def extract_features_hist(cols):
  crr = cols[0]
  crrlst= crr.split(',')
  test_list = list(filter(None, crrlst))
  if not test_list:
    return 0
  else:
    res_int = [int(float(i)) for i in test_list]
    crrmedian = statistics.mean(res_int)
    return crrmedian 

In [ ]:
bur_test['AMT OVERDUE - HIST_Mean'] = bur_test[['AMT OVERDUE - HIST']].apply(extract_features_hist,axis=1)
bur_test['AMT PAID - HIST_Mean'] = bur_test[['AMT PAID - HIST']].apply(extract_features_hist,axis=1)

In [ ]:
bur_test.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,DPD_Hist_Median,CUR BAL-Hist_Median,AMT OVERDUE - HIST_Median,AMT PAID - HIST_Median,DPD_Hist_Mean,CUR BAL-Hist_Mean,AMT OVERDUE - HIST_Mean,AMT PAID - HIST_Mean
0,4,False,PRIMARY,Credit Card,CCC,2019-11-17,Individual,Active,2019-09-26,NaN,NaT,"10,000",588,0,589,NaN,0,0,NaN,"20191117,",000,"589,","0,","0,",0,0.0,589.0,0.0,0.0,0.000000,5.890000e+02,0.000000,0.0
1,4,False,PRIMARY,Housing Loan,HFC,2019-10-31,Joint,Active,2017-06-21,NaN,2019-10-15,NaN,2500000,30797,2107227,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"2107227,2122196,2136877,2151448,2165910,218026...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120,0.0,2306032.0,0.0,0.0,2.793103,2.308055e+06,2286.827586,0.0
2,4,False,PRIMARY,Property Loan,HFC,2019-10-31,Joint,Active,2017-06-27,NaN,2019-10-15,NaN,99000,1223,83458,F03,0,0,NaN,"20191031,20190930,20190831,20190731,20190630,2...",0000000000000000000000000000000000000000000000...,"83458,84051,84632,85209,85781,86349,86913,8747...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,",120,0.0,91325.0,0.0,0.0,2.068966,9.132148e+04,6.344828,0.0
3,4,True,PRIMARY,Tractor Loan,NBF,2015-10-31,Individual,Closed,2011-12-29,2015-10-13 00:00:00,NaT,NaN,365000,0,0,NaN,0,0,NaN,"20151031,20150901,20150831,20150731,20150630,2...",000DDD0270000000000000000000000000000000000000...,"0,,21548,32097,42500,52758,62874,72850,82688,9...","0,,11000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,",0,0.0,173870.0,0.0,0.0,0.750000,1.674939e+05,314.285714,0.0
4,5,False,PRIMARY,Gold Loan,PRB,2017-08-31,Individual,Closed,2016-05-04,2017-06-15 00:00:00,2017-06-15,NaN,25801,0,0,NaN,0,0,NaN,"20170630,20170501,20170401,20170331,20170201,2...",000DDDDDD000DDDDDD000DDDDDD000DDDDDD000,"0,,,25846,,,25846,,,25846,,,25846,","0,,,0,,,0,,,0,,,0",",,,,,,,,,,,,,,",0,0.0,25846.0,0.0,0.0,0.000000,2.067680e+04,0.000000,0.0


In [ ]:
test_data_bur_cat_feat.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0,4,False,3,"[False, True]",PRIMARY,4,[PRIMARY],4,Tractor Loan,1,"[Credit Card, Housing Loan, Property Loan, Tra...",3,HFC,2,"[CCC, HFC, NBF]",2,Individual,2,"[Individual, Joint]",2,Active,3,"[Active, Closed]"
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0,8,False,7,"[False, True]",PRIMARY,8,[PRIMARY],5,Gold Loan,3,"[Gold Loan, Consumer Loan, Personal Loan, Trac...",4,NBF,3,"[PRB, NBF, HFC, MFI]",2,Individual,6,"[Individual, Joint]",2,Closed,6,"[Closed, Active]"
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5,12,False,10,"[False, True]",PRIMARY,12,[PRIMARY],4,Auto Loan (Personal),7,"[Auto Loan (Personal), Tractor Loan, Gold Loan...",2,NBF,11,"[NBF, PRB]",3,Individual,6,"[Guarantor, Individual, Joint]",2,Closed,11,"[Closed, Active]"
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Business Loan Priority Sector Agriculture,1,"[Business Loan Priority Sector Agriculture, T...",2,NBF,1,"[NAB, NBF]",1,Individual,2,[Individual],2,Active,1,"[Active, Closed]"
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Auto Loan (Personal),1,"[Auto Loan (Personal), Tractor Loan]",1,NBF,2,[NBF],2,Guarantor,1,"[Guarantor, Individual]",2,Active,1,"[Active, Closed]"


In [ ]:
test_data_bur_cat_feat.shape

(14745, 48)

In [ ]:
bur_test_groupby = bur_test.groupby('ID')
bur_test_groupby

In [ ]:
def extract_mean_values(bur_test_groupby,features,test_data_bur_cat_feat):
  for feature in features:
    feat_median = bur_test_groupby[feature].mean()
    d=feat_median.to_frame().reset_index()
    d.columns = ['ID','{}_Mean'.format(feature)]
    test_data_bur_cat_feat = pd.merge(test_data_bur_cat_feat,d,on='ID')
  return test_data_bur_cat_feat

test_data_bur_demo_final_mean = extract_mean_values(bur_test_groupby,[
                                                             'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
                                                             'OVERDUE-AMT', 'WRITE-OFF-AMT','TENURE',
                                                             'DPD_Hist_Mean', 'CUR BAL-Hist_Mean',
                                                             'AMT OVERDUE - HIST_Mean', 'AMT PAID - HIST_Mean'],test_data_bur_cat_feat)

In [ ]:
test_data_bur_demo_final_mean = test_data_bur_demo_final_mean.drop(['DISBURSED-AMT/HIGH CREDIT_Median',
       'INSTALLMENT-AMT_Median', 'CURRENT-BAL_Median', 'OVERDUE-AMT_Median',
       'WRITE-OFF-AMT_Median', 'TENURE_Median'],axis=1)
test_data_bur_demo_final_mean.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,False,3,"[False, True]",PRIMARY,4,[PRIMARY],4,Tractor Loan,1,"[Credit Card, Housing Loan, Property Loan, Tra...",3,HFC,2,"[CCC, HFC, NBF]",2,Individual,2,"[Individual, Joint]",2,Active,3,"[Active, Closed]",741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,False,7,"[False, True]",PRIMARY,8,[PRIMARY],5,Gold Loan,3,"[Gold Loan, Consumer Loan, Personal Loan, Trac...",4,NBF,3,"[PRB, NBF, HFC, MFI]",2,Individual,6,"[Individual, Joint]",2,Closed,6,"[Closed, Active]",242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,False,10,"[False, True]",PRIMARY,12,[PRIMARY],4,Auto Loan (Personal),7,"[Auto Loan (Personal), Tractor Loan, Gold Loan...",2,NBF,11,"[NBF, PRB]",3,Individual,6,"[Guarantor, Individual, Joint]",2,Closed,11,"[Closed, Active]",269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Business Loan Priority Sector Agriculture,1,"[Business Loan Priority Sector Agriculture, T...",2,NBF,1,"[NAB, NBF]",1,Individual,2,[Individual],2,Active,1,"[Active, Closed]",400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Auto Loan (Personal),1,"[Auto Loan (Personal), Tractor Loan]",1,NBF,2,[NBF],2,Guarantor,1,"[Guarantor, Individual]",2,Active,1,"[Active, Closed]",465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0


In [ ]:
test_data_bur_demo_final_mean.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,False,3,"[False, True]",PRIMARY,4,[PRIMARY],4,Tractor Loan,1,"[Credit Card, Housing Loan, Property Loan, Tra...",3,HFC,2,"[CCC, HFC, NBF]",2,Individual,2,"[Individual, Joint]",2,Active,3,"[Active, Closed]",741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,False,7,"[False, True]",PRIMARY,8,[PRIMARY],5,Gold Loan,3,"[Gold Loan, Consumer Loan, Personal Loan, Trac...",4,NBF,3,"[PRB, NBF, HFC, MFI]",2,Individual,6,"[Individual, Joint]",2,Closed,6,"[Closed, Active]",242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,False,10,"[False, True]",PRIMARY,12,[PRIMARY],4,Auto Loan (Personal),7,"[Auto Loan (Personal), Tractor Loan, Gold Loan...",2,NBF,11,"[NBF, PRB]",3,Individual,6,"[Guarantor, Individual, Joint]",2,Closed,11,"[Closed, Active]",269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Business Loan Priority Sector Agriculture,1,"[Business Loan Priority Sector Agriculture, T...",2,NBF,1,"[NAB, NBF]",1,Individual,2,[Individual],2,Active,1,"[Active, Closed]",400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,2,True,1,"[False, True]",PRIMARY,2,[PRIMARY],2,Auto Loan (Personal),1,"[Auto Loan (Personal), Tractor Loan]",1,NBF,2,[NBF],2,Guarantor,1,"[Guarantor, Individual]",2,Active,1,"[Active, Closed]",465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0


In [ ]:
test_data_bur_demo_final_mean.shape

(14745, 52)

In [ ]:
test_data_bur_demo_final_mean.to_excel('test_bur_demo_combined_with_hist_mean.xlsx')

### Cross Checking Mean values of Train dataset 

In [ ]:
train_data_bur = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/train_data_bur.xlsx')
train_data_bur = train_data_bur.iloc[:,1:]
bur_train = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/bureau_data_train_manipulated.xlsx')
bur_train = bur_train.iloc[:,1:]

In [ ]:
import statistics
def extract_features_dpd(cols):
  dpdd = cols[0]
  splitted_string= [dpdd[i:i+3] for i in range(0, len(dpdd), 3)]
  resD = [ele.replace('DDD','000') for ele in splitted_string]
  res_final = [ele.replace('XXX','000') for ele in resD]
  res_int = [int(float(i)) for i in res_final]
  dpdmedian = statistics.mean(res_int)
  return dpdmedian

In [ ]:
bur_train['DPD - HIST'] = bur_train['DPD - HIST'].fillna('0')
bur_train.loc[bur_train['DPD - HIST'].str.contains('E'),'DPD - HIST']='0'
bur_train['DPD_Hist_Mean'] = bur_train[['DPD - HIST']].apply(extract_features_dpd,axis=1)

In [ ]:
import statistics
def extract_features_hist(cols):
  crr = cols[0]
  crrlst= crr.split(',')
  test_list = list(filter(None, crrlst))
  if not test_list:
    return 0
  else:
    res_int = [int(float(i)) for i in test_list]
    crrmedian = statistics.mean(res_int)
    return crrmedian 

In [ ]:
bur_train['CUR BAL - HIST'] = bur_train['CUR BAL - HIST'].fillna('0')
bur_train['AMT OVERDUE - HIST'] = bur_train['AMT OVERDUE - HIST'].fillna('0')
bur_train['AMT PAID - HIST'] = bur_train['AMT PAID - HIST'].fillna('0')

In [ ]:
bur_train['AMT OVERDUE - HIST_Mean'] = bur_train[['AMT OVERDUE - HIST']].apply(extract_features_hist,axis=1)
bur_train['AMT PAID - HIST_Mean'] = bur_train[['AMT PAID - HIST']].apply(extract_features_hist,axis=1)

In [ ]:
import statistics
def extract_features_curr_bal_hist(cols):
  crr = cols[0]
  crrlst= crr.split(',')
  test_list = list(filter(None, crrlst))
  if not test_list:
    return 0
  else:
    res_int = [int(i) for i in test_list]
    crrmedian = statistics.mean(res_int)
    return crrmedian 

bur_train['CUR BAL-Hist_Mean'] = bur_train[['CUR BAL - HIST']].apply(extract_features_curr_bal_hist,axis=1)

In [ ]:
bur_train_groupby = bur_train.groupby('ID')

In [ ]:
def extract_mean_values(bur_train_groupby,features,train_data_bur):
  for feature in features:
    feat_median = bur_train_groupby[feature].mean()
    d=feat_median.to_frame().reset_index()
    d.columns = ['ID','{}_Mean'.format(feature)]
    train_data_bur = pd.merge(train_data_bur,d,on='ID')
  return train_data_bur

train_data_bur_demo_final_mean = extract_mean_values(bur_train_groupby,[
                                                             'DISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT', 'CURRENT-BAL',
                                                             'OVERDUE-AMT', 'WRITE-OFF-AMT','TENURE',
                                                             'DPD_Hist_Mean', 'CUR BAL-Hist_Mean',
                                                             'AMT OVERDUE - HIST_Mean', 'AMT PAID - HIST_Mean'],train_data_bur)

In [ ]:
train_data_bur_demo_final_mean.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,275000.0,0.0,0.0,0.0,0.0,0.0,2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,700000.0,0.0,52421.0,0.0,0.0,35.0,1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,44000.0,0.0,0.0,0.0,0.0,0.0,1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,315000.0,0.0,0.0,0.0,0.0,5.0,3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,200000.0,0.0,6989.0,0.0,0.0,42.0,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


In [ ]:
train_data_bur_demo_final_mean.iloc[2]['CUR BAL-Hist_Mean_Mean']

154949.5020018999

##### String Maniulation on Test Data 

In [ ]:
test_combined_median = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/test_bur_demo_combined_with_hist_median.xlsx')
test_combined_median.head()

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DPD_Hist_Median_Median,CUR BAL-Hist_Median_Median,AMT OVERDUE - HIST_Median_Median,AMT PAID - HIST_Median_Median
0,0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0,4,False,3,[False True],PRIMARY,4,['PRIMARY'],4,Tractor Loan,1,['Credit Card' 'Housing Loan' 'Property Loan' ...,3,HFC,2,['CCC' 'HFC' 'NBF'],2,Individual,2,['Individual' 'Joint'],2,Active,3,['Active' 'Closed'],0.0,132597.5,0.0,0.0
1,1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0,8,False,7,[False True],PRIMARY,8,['PRIMARY'],5,Gold Loan,3,['Gold Loan' 'Consumer Loan' 'Personal Loan' '...,4,NBF,3,['PRB' 'NBF' 'HFC' 'MFI'],2,Individual,6,['Individual' 'Joint'],2,Closed,6,['Closed' 'Active'],0.0,98453.5,0.0,0.0
2,2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5,12,False,10,[False True],PRIMARY,12,['PRIMARY'],4,Auto Loan (Personal),7,['Auto Loan (Personal)' 'Tractor Loan' 'Gold L...,2,NBF,11,['NBF' 'PRB'],3,Individual,6,['Guarantor' 'Individual' 'Joint'],2,Closed,11,['Closed' 'Active'],0.0,22018.0,0.0,0.0
3,3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0,2,True,1,[False True],PRIMARY,2,['PRIMARY'],2,Business Loan Priority Sector Agriculture,1,['Business Loan Priority Sector Agriculture' ...,2,NBF,1,['NAB' 'NBF'],1,Individual,2,['Individual'],2,Active,1,['Active' 'Closed'],0.0,139003.0,0.0,0.0
4,4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0,2,True,1,[False True],PRIMARY,2,['PRIMARY'],2,Auto Loan (Personal),1,['Auto Loan (Personal)' 'Tractor Loan'],1,NBF,2,['NBF'],2,Guarantor,1,['Guarantor' 'Individual'],2,Active,1,['Active' 'Closed'],0.0,254296.0,0.0,265000.0


In [ ]:
test_combined_median['SELF-INDICATOR_unique_values'].value_counts()

[False  True]    6791
[ True]          4572
[ True False]    3185
[False]           197
Name: SELF-INDICATOR_unique_values, dtype: int64

In [ ]:
### preprocessing of Self Indicator
def preproc_self_ind(cols):
  t = cols[0]
  rep = {"[": "","]":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

In [ ]:
test_combined_median['SELF-INDICATOR_unique_values'] = test_combined_median[['SELF-INDICATOR_unique_values']].apply(preproc_self_ind,axis=1)
test_combined_median['SELF-INDICATOR_unique_values'].value_counts()

False  True    6791
True           4572
True False     3185
False           197
Name: SELF-INDICATOR_unique_values, dtype: int64

In [ ]:
## Convert False True to True False
def preproc_self_ind_false_true(cols):
  t = cols[0]
  rep = {"False  True": "True False"} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

In [ ]:
test_combined_median['SELF-INDICATOR_unique_values'] = test_combined_median[['SELF-INDICATOR_unique_values']].apply(preproc_self_ind_false_true,axis=1)
test_combined_median['SELF-INDICATOR_unique_values'].value_counts()

True False    9976
True          4572
False          197
Name: SELF-INDICATOR_unique_values, dtype: int64

### Match Type Unique Values 

In [ ]:
test_combined_median['MATCH-TYPE_unique_values'].value_counts()

['PRIMARY']                14734
['SECONDARY']                  9
['PRIMARY' 'SECONDARY']        2
Name: MATCH-TYPE_unique_values, dtype: int64

In [ ]:
def preproc_self_match_type(cols):
  t = cols[0]
  rep = {"'": "","[":"","]":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

In [ ]:
test_combined_median['MATCH-TYPE_unique_values'] = test_combined_median[['MATCH-TYPE_unique_values']].apply(preproc_self_match_type,axis=1)
test_combined_median['MATCH-TYPE_unique_values'].value_counts()

PRIMARY              14734
SECONDARY                9
PRIMARY SECONDARY        2
Name: MATCH-TYPE_unique_values, dtype: int64

Other Features


In [ ]:
def preproc_acc_type_unique(cols):
  t = cols[0]
  rep = {"[":"","]":"","\n":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  text1 = text.replace("' '",'|')
  text2 = text1.replace("'","")
  splitt = text2.split('|')
  final = sorted(splitt)
  final_text = str(final)
  return final_text

In [ ]:
len(test_combined_median['ACCT-TYPE_unique_values'].unique())

3242

In [ ]:
test_combined_median['ACCT-TYPE_unique_values'] = test_combined_median[['ACCT-TYPE_unique_values']].apply(preproc_acc_type_unique,axis=1)

In [ ]:
len(test_combined_median['ACCT-TYPE_unique_values'].unique())

1743

In [ ]:
len(test_combined_median['CONTRIBUTOR-TYPE_unique_values'].unique())

379

In [ ]:
test_combined_median['CONTRIBUTOR-TYPE_unique_values'] = test_combined_median[['CONTRIBUTOR-TYPE_unique_values']].apply(preproc_acc_type_unique,axis=1)

In [ ]:
len(test_combined_median['CONTRIBUTOR-TYPE_unique_values'].unique())

99

In [ ]:
len(test_combined_median['OWNERSHIP-IND_unique_values'].unique())

37

In [ ]:
test_combined_median['OWNERSHIP-IND_unique_values'] = test_combined_median[['OWNERSHIP-IND_unique_values']].apply(preproc_acc_type_unique,axis=1)

In [ ]:
len(test_combined_median['OWNERSHIP-IND_unique_values'].unique())

13

In [ ]:
len(test_combined_median['ACCOUNT-STATUS_unique_values'].unique())

145

In [ ]:
test_combined_median['ACCOUNT-STATUS_unique_values'] = test_combined_median[['ACCOUNT-STATUS_unique_values']].apply(preproc_acc_type_unique,axis=1)

In [ ]:
len(test_combined_median['ACCOUNT-STATUS_unique_values'].unique())

44

In [ ]:
test_combined_median.head()

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DPD_Hist_Median_Median,CUR BAL-Hist_Median_Median,AMT OVERDUE - HIST_Median_Median,AMT PAID - HIST_Median_Median
0,0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0,4,False,3,True False,PRIMARY,4,PRIMARY,4,Tractor Loan,1,"['Credit Card', 'Housing Loan', 'Property Loan...",3,HFC,2,"['CCC', 'HFC', 'NBF']",2,Individual,2,"['Individual', 'Joint']",2,Active,3,"['Active', 'Closed']",0.0,132597.5,0.0,0.0
1,1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0,8,False,7,True False,PRIMARY,8,PRIMARY,5,Gold Loan,3,"['Consumer Loan', 'Gold Loan', 'Housing Loan',...",4,NBF,3,"['HFC', 'MFI', 'NBF', 'PRB']",2,Individual,6,"['Individual', 'Joint']",2,Closed,6,"['Active', 'Closed']",0.0,98453.5,0.0,0.0
2,2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5,12,False,10,True False,PRIMARY,12,PRIMARY,4,Auto Loan (Personal),7,"['Auto Loan (Personal)', 'Gold Loan', 'Persona...",2,NBF,11,"['NBF', 'PRB']",3,Individual,6,"['Guarantor', 'Individual', 'Joint']",2,Closed,11,"['Active', 'Closed']",0.0,22018.0,0.0,0.0
3,3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0,2,True,1,True False,PRIMARY,2,PRIMARY,2,Business Loan Priority Sector Agriculture,1,"['Business Loan Priority Sector Agriculture',...",2,NBF,1,"['NAB', 'NBF']",1,Individual,2,['Individual'],2,Active,1,"['Active', 'Closed']",0.0,139003.0,0.0,0.0
4,4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0,2,True,1,True False,PRIMARY,2,PRIMARY,2,Auto Loan (Personal),1,"['Auto Loan (Personal)', 'Tractor Loan']",1,NBF,2,['NBF'],2,Guarantor,1,"['Guarantor', 'Individual']",2,Active,1,"['Active', 'Closed']",0.0,254296.0,0.0,265000.0


In [ ]:
test_combined_median = test_combined_median.iloc[:,1:]
test_combined_median

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DPD_Hist_Median_Median,CUR BAL-Hist_Median_Median,AMT OVERDUE - HIST_Median_Median,AMT PAID - HIST_Median_Median
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,232000.0,611.5,42023.5,0.0,0.0,60.0,4,False,3,True False,PRIMARY,4,PRIMARY,4,Tractor Loan,1,"['Credit Card', 'Housing Loan', 'Property Loan...",3,HFC,2,"['CCC', 'HFC', 'NBF']",2,Individual,2,"['Individual', 'Joint']",2,Active,3,"['Active', 'Closed']",0.0,132597.50,0.0,0.0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,106008.0,0.0,0.0,0.0,0.0,0.0,8,False,7,True False,PRIMARY,8,PRIMARY,5,Gold Loan,3,"['Consumer Loan', 'Gold Loan', 'Housing Loan',...",4,NBF,3,"['HFC', 'MFI', 'NBF', 'PRB']",2,Individual,6,"['Individual', 'Joint']",2,Closed,6,"['Active', 'Closed']",0.0,98453.50,0.0,0.0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,230000.0,0.0,0.0,0.0,0.0,9.5,12,False,10,True False,PRIMARY,12,PRIMARY,4,Auto Loan (Personal),7,"['Auto Loan (Personal)', 'Gold Loan', 'Persona...",2,NBF,11,"['NBF', 'PRB']",3,Individual,6,"['Guarantor', 'Individual', 'Joint']",2,Closed,11,"['Active', 'Closed']",0.0,22018.00,0.0,0.0
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,400000.0,0.0,63796.5,0.0,0.0,0.0,2,True,1,True False,PRIMARY,2,PRIMARY,2,Business Loan Priority Sector Agriculture,1,"['Business Loan Priority Sector Agriculture',...",2,NBF,1,"['NAB', 'NBF']",1,Individual,2,['Individual'],2,Active,1,"['Active', 'Closed']",0.0,139003.00,0.0,0.0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,465000.0,8000.0,58100.0,0.0,0.0,23.0,2,True,1,True False,PRIMARY,2,PRIMARY,2,Auto Loan (Personal),1,"['Auto Loan (Personal)', 'Tractor Loan']",1,NBF,2,['NBF'],2,Guarantor,1,"['Guarantor', 'Individual']",2,Active,1,"['Active', 'Closed']",0.0,254296.00,0.0,265000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14740,0.255776,143,35,530016,419616.0,14723.0,1046,22094,40,35833.33,0.533554,122001,0,1,0,1,1,1,143396,228690.0,0.0,0.0,0.0,0.0,0.0,39,True,23,True False,PRIMARY,39,PRIMARY,6,Tractor Loan,23,"['Auto Loan (Personal)', 'Business Loan Genera...",4,NBF,28,"['FRB', 'HFC', 'NBF', 'PRB']",3,Guarantor,23,"['Guarantor', 'Individual', 'Joint']",2,Closed,32,"['Active', 'Closed']",0.0,111410.50,0.0,0.0
14741,0.255776,32,36,595000,446500.0,15491.0,2721,61196,40,35833.33,0.436771,560020,0,1,0,1,1,1,143397,1850000.0,0.0,0.0,0.0,0.0,0.0,6,False,4,True False,PRIMARY,6,PRIMARY,4,Business Loan General,2,"['Business Loan General', 'Housing Loan', 'Pro...",3,NBF,4,"['FRB', 'NAB', 'NBF']",2,Joint,5,"['Individual', 'Joint']",2,Closed,5,"['Active', 'Closed']",0.0,117150.00,0.0,0.0
14742,0.255776,32,36,595000,446500.0,15491.0,2721,61196,40,35833.33,0.436771,560020,0,1,

In [ ]:
test_combined_median.dtypes

PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M                                         int64
ID                                            int64
DISBURSED-AM

In [ ]:
test_combined_median['SELF-INDICATOR_Top_value'] = test_combined_median['SELF-INDICATOR_Top_value'].astype('object')
test_combined_median.dtypes

PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M                                         int64
ID                                            int64
DISBURSED-AM

In [ ]:
test_combined_median.to_excel('test_bur_demo_median_updated_string_manipulation.xlsx')

### String Manipulation of Mean 

In [ ]:
train_combined_median = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/train_bur_demo_combined_with_hist_mean.xlsx')
train_combined_median.head()

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,9,True,5,[False True],PRIMARY,9,['PRIMARY'],4,Auto Loan (Personal),5,['Overdraft' 'Auto Loan (Personal)' 'Tractor L...,2,NBF,5,['NAB' 'NBF'],1,Individual,9,['Individual'],3,Closed,5,['Delinquent' 'Active' 'Closed'],2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,13,False,12,[False True],PRIMARY,13,['PRIMARY'],7,Commercial Vehicle Loan,4,['Overdraft' 'Housing Loan' 'Business Loan Gen...,2,NBF,7,['PRB' 'NBF'],2,Individual,9,['Individual' 'Guarantor'],2,Active,7,['Closed' 'Active'],1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,31,False,29,[ True False],PRIMARY,31,['PRIMARY'],8,Gold Loan,8,['Two-Wheeler Loan' 'Credit Card' 'Personal Lo...,5,PRB,13,['NBF' 'PRB' 'CCC' 'COP' 'NAB'],3,Individual,26,['Individual' 'Joint' 'Guarantor'],3,Closed,19,['Closed' 'Active' 'Delinquent'],1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,4,True,2,[ True False],PRIMARY,4,['PRIMARY'],2,Tractor Loan,3,['Tractor Loan' 'Personal Loan'],1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,['Closed' 'Active'],3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,7,False,6,[False True],PRIMARY,7,['PRIMARY'],3,Tractor Loan,4,['Commercial Vehicle Loan' 'Personal Loan' 'Tr...,3,NBF,4,['NBF' 'NAB' 'PRB'],3,Individual,3,['Guarantor' 'Individual' 'Joint'],4,Active,3,['SUIT FILED (WILFUL DEFAULT)' 'Active' 'Delin...,2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524


In [ ]:
### preprocessing of Self Indicator
def preproc_self_ind(cols):
  t = cols[0]
  rep = {"[": "","]":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

train_combined_median['SELF-INDICATOR_unique_values'] = train_combined_median[['SELF-INDICATOR_unique_values']].apply(preproc_self_ind,axis=1)

In [ ]:
## Convert False True to True False
def preproc_self_ind_false_true(cols):
  t = cols[0]
  rep = {"False  True": "True False"} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

train_combined_median['SELF-INDICATOR_unique_values'] = train_combined_median[['SELF-INDICATOR_unique_values']].apply(preproc_self_ind_false_true,axis=1)
train_combined_median['SELF-INDICATOR_unique_values'].value_counts()

True False    75526
True          51010
False          2118
Name: SELF-INDICATOR_unique_values, dtype: int64

In [ ]:
def preproc_self_match_type(cols):
  t = cols[0]
  rep = {"'": "","[":"","]":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

train_combined_median['MATCH-TYPE_unique_values'] = train_combined_median[['MATCH-TYPE_unique_values']].apply(preproc_self_match_type,axis=1)
train_combined_median['MATCH-TYPE_unique_values'].value_counts()

PRIMARY              128576
PRIMARY SECONDARY        47
SECONDARY                31
Name: MATCH-TYPE_unique_values, dtype: int64

In [ ]:
def preproc_acc_type_unique(cols):
  t = cols[0]
  rep = {"[":"","]":"","\n":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  text1 = text.replace("' '",'|')
  text2 = text1.replace("'","")
  splitt = text2.split('|')
  final = sorted(splitt)
  final_text = str(final)
  return final_text

train_combined_median['ACCT-TYPE_unique_values'] = train_combined_median[['ACCT-TYPE_unique_values']].apply(preproc_acc_type_unique,axis=1)
len(train_combined_median['ACCT-TYPE_unique_values'].unique())

7782

In [ ]:
train_combined_median['CONTRIBUTOR-TYPE_unique_values'] = train_combined_median[['CONTRIBUTOR-TYPE_unique_values']].apply(preproc_acc_type_unique,axis=1)
len(train_combined_median['CONTRIBUTOR-TYPE_unique_values'].unique())

182

In [ ]:
train_combined_median['OWNERSHIP-IND_unique_values'] = train_combined_median[['OWNERSHIP-IND_unique_values']].apply(preproc_acc_type_unique,axis=1)
len(train_combined_median['OWNERSHIP-IND_unique_values'].unique())

18

In [ ]:
train_combined_median['ACCOUNT-STATUS_unique_values'] = train_combined_median[['ACCOUNT-STATUS_unique_values']].apply(preproc_acc_type_unique,axis=1)
len(train_combined_median['ACCOUNT-STATUS_unique_values'].unique())

91

In [ ]:
train_combined_median.head(10)

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,9,True,5,True False,PRIMARY,9,PRIMARY,4,Auto Loan (Personal),5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,"['NAB', 'NBF']",1,Individual,9,['Individual'],3,Closed,5,"['Active', 'Closed', 'Delinquent']",2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,13,False,12,True False,PRIMARY,13,PRIMARY,7,Commercial Vehicle Loan,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,"['NBF', 'PRB']",2,Individual,9,"['Guarantor', 'Individual']",2,Active,7,"['Active', 'Closed']",1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,31,False,29,True False,PRIMARY,31,PRIMARY,8,Gold Loan,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,"['CCC', 'COP', 'NAB', 'NBF', 'PRB']",3,Individual,26,"['Guarantor', 'Individual', 'Joint']",3,Closed,19,"['Active', 'Closed', 'Delinquent']",1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,4,True,2,True False,PRIMARY,4,PRIMARY,2,Tractor Loan,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,"['Active', 'Closed']",3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,7,False,6,True False,PRIMARY,7,PRIMARY,3,Tractor Loan,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,"['NAB', 'NBF', 'PRB']",3,Individual,3,"['Guarantor', 'Individual', 'Joint']",4,Active,3,"['Active', 'Closed', 'Delinquent', 'SUIT FILED...",2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524
5,5,0.168888,5,48,400000,280000.0,53000.0,1019,54689,40,42083.33,0.269824,493885,0,0,1,0,1,1,1,9,1,True,1,True,PRIMARY,1,PRIMARY,1,Tractor Loan,1,['Tractor Loan'],1,NBF,1,['NBF'],1,Individual,1,['Individual'],1,Closed,1,['Closed'],2.800000e+05,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.439588e+05,0.000000,0.000000
6,6,0.186877,5,48,716000,450000.0,2000.0,1019,54689,23,46221.00,0.269824,493889,0,0,1,0,1,1,1,10,3,False,2,True False,PRIMARY,3,PRIMARY,2,Tractor Loan,2,"['Commercial Vehicle Loan', 'Tractor Loan']",1,NBF,3,['NBF'],2,Individual,2,"['Guarantor', 'Individual']",2,Closed,2,"['Closed', 'Delinquent']",3.880893e+05,1150.000000,94574.333333,3635.666667,0.0,31.333333,16.879630,1.923314e+05,839.641026,3632.051282
7,7,0.186877,5,48,60

In [ ]:
train_combined_median = train_combined_median.iloc[:,1:]
train_combined_median

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,9,True,5,True False,PRIMARY,9,PRIMARY,4,Auto Loan (Personal),5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,"['NAB', 'NBF']",1,Individual,9,['Individual'],3,Closed,5,"['Active', 'Closed', 'Delinquent']",2.445947e+05,1037.666667,68725.111111,4208.111111,0.0,13.333333,7.570988,1.333202e+05,4668.511111,0.000000
1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,13,False,12,True False,PRIMARY,13,PRIMARY,7,Commercial Vehicle Loan,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,"['NBF', 'PRB']",2,Individual,9,"['Guarantor', 'Individual']",2,Active,7,"['Active', 'Closed']",1.393622e+06,13404.846154,796112.076923,0.000000,0.0,43.461538,1.209402,1.029354e+06,133.876068,136337.034188
2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,31,False,29,True False,PRIMARY,31,PRIMARY,8,Gold Loan,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,"['CCC', 'COP', 'NAB', 'NBF', 'PRB']",3,Individual,26,"['Guarantor', 'Individual', 'Joint']",3,Closed,19,"['Active', 'Closed', 'Delinquent']",1.196248e+05,3078.580645,66618.225806,3338.387097,0.0,14.645161,2.404642,1.549495e+05,1761.551260,5506.888724
3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,4,True,2,True False,PRIMARY,4,PRIMARY,2,Tractor Loan,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,"['Active', 'Closed']",3.181382e+05,1392.750000,156000.000000,0.000000,0.0,8.250000,0.000000,2.810307e+05,0.000000,0.000000
4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,7,False,6,True False,PRIMARY,7,PRIMARY,3,Tractor Loan,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,"['NAB', 'NBF', 'PRB']",3,Individual,3,"['Guarantor', 'Individual', 'Joint']",4,Active,3,"['Active', 'Closed', 'Delinquent', 'SUIT FILED...",2.893166e+05,1248.285714,139159.857143,5026.428571,0.0,33.714286,8.797619,2.471944e+05,2114.394236,5653.809524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128649,0.668451,424,24,470000,265601.0,76800.0,1568,48879,25,65333.33,0.533554,131403,3,1,0,0,1,1,1,143390,2,True,1,True False,PRIMARY,2,PRIMARY,2,Property Loan,1,"['Property Loan', 'Tractor Loan']",2,NBF,1,"['NBF', 'PRB']",1,Individual,2,['Individual'],1,Active,2,['Active'],1.043800e+06,0.000000,933815.000000,0.000000,0.0,60.500000,0.000000,9.912944e+05,0.000000,0.000000
128650,0.668451,424,24,460000,275630.0,80100.0,1568,48879,25,83333.33,0.533554,131403,0,1,0,0,1,1,1,143391,1,True,1,True,PRIMARY,1,PRIMARY,1,Tractor Loan,1,['Tractor Loan'],1,NBF,1,[

In [ ]:
train_combined_median.dtypes

PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
top_up_class                                  int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M                                         int64
ID          

In [ ]:
train_combined_median['SELF-INDICATOR_Top_value'] = train_combined_median['SELF-INDICATOR_Top_value'].astype('object')

In [ ]:
train_combined_median.dtypes

PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
top_up_class                                  int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M                                         int64
ID          

In [ ]:
train_combined_median.to_excel('train_bur_demo_mean_updated_string_manipulation.xlsx')

### test data String manupulation MEAN

In [ ]:
test_combined_median = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/test_bur_demo_combined_with_hist_mean.xlsx')
test_combined_median.head()

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,False,3,[False True],PRIMARY,4,['PRIMARY'],4,Tractor Loan,1,['Credit Card' 'Housing Loan' 'Property Loan' ...,3,HFC,2,['CCC' 'HFC' 'NBF'],2,Individual,2,['Individual' 'Joint'],2,Active,3,['Active' 'Closed'],741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0
1,1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,False,7,[False True],PRIMARY,8,['PRIMARY'],5,Gold Loan,3,['Gold Loan' 'Consumer Loan' 'Personal Loan' '...,4,NBF,3,['PRB' 'NBF' 'HFC' 'MFI'],2,Individual,6,['Individual' 'Joint'],2,Closed,6,['Closed' 'Active'],242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0
2,2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,False,10,[False True],PRIMARY,12,['PRIMARY'],4,Auto Loan (Personal),7,['Auto Loan (Personal)' 'Tractor Loan' 'Gold L...,2,NBF,11,['NBF' 'PRB'],3,Individual,6,['Guarantor' 'Individual' 'Joint'],2,Closed,11,['Closed' 'Active'],269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0
3,3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,True,1,[False True],PRIMARY,2,['PRIMARY'],2,Business Loan Priority Sector Agriculture,1,['Business Loan Priority Sector Agriculture' ...,2,NBF,1,['NAB' 'NBF'],1,Individual,2,['Individual'],2,Active,1,['Active' 'Closed'],400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0
4,4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,2,True,1,[False True],PRIMARY,2,['PRIMARY'],2,Auto Loan (Personal),1,['Auto Loan (Personal)' 'Tractor Loan'],1,NBF,2,['NBF'],2,Guarantor,1,['Guarantor' 'Individual'],2,Active,1,['Active' 'Closed'],465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0


In [ ]:
### preprocessing of Self Indicator
def preproc_self_ind(cols):
  t = cols[0]
  rep = {"[": "","]":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

test_combined_median['SELF-INDICATOR_unique_values'] = test_combined_median[['SELF-INDICATOR_unique_values']].apply(preproc_self_ind,axis=1)
test_combined_median['SELF-INDICATOR_unique_values'].value_counts()

False  True    6791
True           4572
True False     3185
False           197
Name: SELF-INDICATOR_unique_values, dtype: int64

In [ ]:
## Convert False True to True False
def preproc_self_ind_false_true(cols):
  t = cols[0]
  rep = {"False  True": "True False"} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text

test_combined_median['SELF-INDICATOR_unique_values'] = test_combined_median[['SELF-INDICATOR_unique_values']].apply(preproc_self_ind_false_true,axis=1)
test_combined_median['SELF-INDICATOR_unique_values'].value_counts()

True False    9976
True          4572
False          197
Name: SELF-INDICATOR_unique_values, dtype: int64

In [ ]:
def preproc_self_match_type(cols):
  t = cols[0]
  rep = {"'": "","[":"","]":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  return text
test_combined_median['MATCH-TYPE_unique_values'] = test_combined_median[['MATCH-TYPE_unique_values']].apply(preproc_self_match_type,axis=1)
test_combined_median['MATCH-TYPE_unique_values'].value_counts()

PRIMARY              14734
SECONDARY                9
PRIMARY SECONDARY        2
Name: MATCH-TYPE_unique_values, dtype: int64

In [ ]:
def preproc_acc_type_unique(cols):
  t = cols[0]
  rep = {"[":"","]":"","\n":""} # define desired replacements here
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  text = pattern.sub(lambda m: rep[re.escape(m.group(0))], t).strip()
  text1 = text.replace("' '",'|')
  text2 = text1.replace("'","")
  splitt = text2.split('|')
  final = sorted(splitt)
  final_text = str(final)
  return final_text

test_combined_median['ACCT-TYPE_unique_values'] = test_combined_median[['ACCT-TYPE_unique_values']].apply(preproc_acc_type_unique,axis=1)
len(test_combined_median['ACCT-TYPE_unique_values'].unique())

1743

In [ ]:
test_combined_median['CONTRIBUTOR-TYPE_unique_values'] = test_combined_median[['CONTRIBUTOR-TYPE_unique_values']].apply(preproc_acc_type_unique,axis=1)
len(test_combined_median['CONTRIBUTOR-TYPE_unique_values'].unique())

99

In [ ]:
test_combined_median['OWNERSHIP-IND_unique_values'] = test_combined_median[['OWNERSHIP-IND_unique_values']].apply(preproc_acc_type_unique,axis=1)
len(test_combined_median['OWNERSHIP-IND_unique_values'].unique())

13

In [ ]:
test_combined_median['ACCOUNT-STATUS_unique_values'] = test_combined_median[['ACCOUNT-STATUS_unique_values']].apply(preproc_acc_type_unique,axis=1)
len(test_combined_median['ACCOUNT-STATUS_unique_values'].unique())

44

In [ ]:
test_combined_median.head()

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,False,3,True False,PRIMARY,4,PRIMARY,4,Tractor Loan,1,"['Credit Card', 'Housing Loan', 'Property Loan...",3,HFC,2,"['CCC', 'HFC', 'NBF']",2,Individual,2,"['Individual', 'Joint']",2,Active,3,"['Active', 'Closed']",741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0
1,1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,False,7,True False,PRIMARY,8,PRIMARY,5,Gold Loan,3,"['Consumer Loan', 'Gold Loan', 'Housing Loan',...",4,NBF,3,"['HFC', 'MFI', 'NBF', 'PRB']",2,Individual,6,"['Individual', 'Joint']",2,Closed,6,"['Active', 'Closed']",242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0
2,2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,False,10,True False,PRIMARY,12,PRIMARY,4,Auto Loan (Personal),7,"['Auto Loan (Personal)', 'Gold Loan', 'Persona...",2,NBF,11,"['NBF', 'PRB']",3,Individual,6,"['Guarantor', 'Individual', 'Joint']",2,Closed,11,"['Active', 'Closed']",269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0
3,3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,True,1,True False,PRIMARY,2,PRIMARY,2,Business Loan Priority Sector Agriculture,1,"['Business Loan Priority Sector Agriculture',...",2,NBF,1,"['NAB', 'NBF']",1,Individual,2,['Individual'],2,Active,1,"['Active', 'Closed']",400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0
4,4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,2,True,1,True False,PRIMARY,2,PRIMARY,2,Auto Loan (Personal),1,"['Auto Loan (Personal)', 'Tractor Loan']",1,NBF,2,['NBF'],2,Guarantor,1,"['Guarantor', 'Individual']",2,Active,1,"['Active', 'Closed']",465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0


In [ ]:
test_combined_median = test_combined_median.iloc[:,1:]
test_combined_median

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,False,3,True False,PRIMARY,4,PRIMARY,4,Tractor Loan,1,"['Credit Card', 'Housing Loan', 'Property Loan...",3,HFC,2,"['CCC', 'HFC', 'NBF']",2,Individual,2,"['Individual', 'Joint']",2,Active,3,"['Active', 'Closed']",7.411470e+05,8005.000000,5.478185e+05,0.0,0.0,60.000000,1.403017,6.418648e+05,651.864532,0.000000
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,False,7,True False,PRIMARY,8,PRIMARY,5,Gold Loan,3,"['Consumer Loan', 'Gold Loan', 'Housing Loan',...",4,NBF,3,"['HFC', 'MFI', 'NBF', 'PRB']",2,Individual,6,"['Individual', 'Joint']",2,Closed,6,"['Active', 'Closed']",2.424766e+05,1282.500000,1.134039e+05,0.0,0.0,22.500000,2.127743,2.115900e+05,789.747186,0.000000
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,False,10,True False,PRIMARY,12,PRIMARY,4,Auto Loan (Personal),7,"['Auto Loan (Personal)', 'Gold Loan', 'Persona...",2,NBF,11,"['NBF', 'PRB']",3,Individual,6,"['Guarantor', 'Individual', 'Joint']",2,Closed,11,"['Active', 'Closed']",2.697801e+05,0.000000,8.566667e+03,0.0,0.0,14.083333,6.326831,6.612230e+04,1864.970541,0.000000
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,True,1,True False,PRIMARY,2,PRIMARY,2,Business Loan Priority Sector Agriculture,1,"['Business Loan Priority Sector Agriculture',...",2,NBF,1,"['NAB', 'NBF']",1,Individual,2,['Individual'],2,Active,1,"['Active', 'Closed']",4.000000e+05,0.000000,6.379650e+04,0.0,0.0,0.000000,0.000000,1.639306e+05,0.000000,0.000000
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,2,True,1,True False,PRIMARY,2,PRIMARY,2,Auto Loan (Personal),1,"['Auto Loan (Personal)', 'Tractor Loan']",1,NBF,2,['NBF'],2,Guarantor,1,"['Guarantor', 'Individual']",2,Active,1,"['Active', 'Closed']",4.650000e+05,8000.000000,5.810000e+04,0.0,0.0,23.000000,5.388889,2.530201e+05,158.800000,265000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14740,0.255776,143,35,530016,419616.0,14723.0,1046,22094,40,35833.33,0.533554,122001,0,1,0,1,1,1,143396,39,True,23,True False,PRIMARY,39,PRIMARY,6,Tractor Loan,23,"['Auto Loan (Personal)', 'Business Loan Genera...",4,NBF,28,"['FRB', 'HFC', 'NBF', 'PRB']",3,Guarantor,23,"['Guarantor', 'Individual', 'Joint']",2,Closed,32,"['Active', 'Closed']",1.292226e+06,3277.128205,3.140004e+05,0.0,0.0,10.179487,2.354701,1.035514e+06,43.868590,32661.437647
14741,0.255776,32,36,595000,446500.0,15491.0,2721,61196,40,35833.33,0.436771,560020,0,1,0,1,1,1,143397,6,False,4,True False,PRIMARY,6,PRIMARY,4,Business Loan General,2,"['Business Loan General', 'Housing Loan', 'Pro...",3,NBF,4,"[

In [ ]:
test_combined_median.dtypes

PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M                                         int64
ID                                            int64
SELF-INDICAT

In [ ]:
test_combined_median['SELF-INDICATOR_Top_value'] = test_combined_median['SELF-INDICATOR_Top_value'].astype('object')
test_combined_median.dtypes

PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M                                         int64
ID                                            int64
SELF-INDICAT

In [ ]:
test_combined_median.head()

,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DISBURSED-AMT/HIGH CREDIT_Mean,INSTALLMENT-AMT_Mean,CURRENT-BAL_Mean,OVERDUE-AMT_Mean,WRITE-OFF-AMT_Mean,TENURE_Mean,DPD_Hist_Mean_Mean,CUR BAL-Hist_Mean_Mean,AMT OVERDUE - HIST_Mean_Mean,AMT PAID - HIST_Mean_Mean
0,0.186877,2,46,480000,365000.0,1000.0,1046,22354,50,32069.00,0.425187,473001,0,1,0,0,1,1,4,4,False,3,True False,PRIMARY,4,PRIMARY,4,Tractor Loan,1,"['Credit Card', 'Housing Loan', 'Property Loan...",3,HFC,2,"['CCC', 'HFC', 'NBF']",2,Individual,2,"['Individual', 'Joint']",2,Active,3,"['Active', 'Closed']",741147.000000,8005.0,547818.500000,0.0,0.0,60.000000,1.403017,641864.840640,651.864532,0.0
1,0.255776,2,45,480000,285000.0,9300.0,1046,22354,35,25000.00,0.425187,473001,0,1,0,0,1,1,5,8,False,7,True False,PRIMARY,8,PRIMARY,5,Gold Loan,3,"['Consumer Loan', 'Gold Loan', 'Housing Loan',...",4,NBF,3,"['HFC', 'MFI', 'NBF', 'PRB']",2,Individual,6,"['Individual', 'Joint']",2,Closed,6,"['Active', 'Closed']",242476.625000,1282.5,113403.875000,0.0,0.0,22.500000,2.127743,211590.023324,789.747186,0.0
2,0.255776,2,48,580000,400000.0,35800.0,1060,65929,37,23333.33,0.425187,473001,0,0,1,1,1,1,6,12,False,10,True False,PRIMARY,12,PRIMARY,4,Auto Loan (Personal),7,"['Auto Loan (Personal)', 'Gold Loan', 'Persona...",2,NBF,11,"['NBF', 'PRB']",3,Individual,6,"['Guarantor', 'Individual', 'Joint']",2,Closed,11,"['Active', 'Closed']",269780.083333,0.0,8566.666667,0.0,0.0,14.083333,6.326831,66122.300453,1864.970541,0.0
3,0.168888,154,36,725000,500000.0,52000.0,1049,68597,55,91666.67,0.269824,493558,1,0,0,1,1,1,25,2,True,1,True False,PRIMARY,2,PRIMARY,2,Business Loan Priority Sector Agriculture,1,"['Business Loan Priority Sector Agriculture',...",2,NBF,1,"['NAB', 'NBF']",1,Individual,2,['Individual'],2,Active,1,"['Active', 'Closed']",400000.000000,0.0,63796.500000,0.0,0.0,0.000000,0.000000,163930.644118,0.000000,0.0
4,0.255776,194,48,617000,400000.0,35400.0,1049,38493,48,12500.00,0.430084,754137,0,0,1,1,1,1,119,2,True,1,True False,PRIMARY,2,PRIMARY,2,Auto Loan (Personal),1,"['Auto Loan (Personal)', 'Tractor Loan']",1,NBF,2,['NBF'],2,Guarantor,1,"['Guarantor', 'Individual']",2,Active,1,"['Active', 'Closed']",465000.000000,8000.0,58100.000000,0.0,0.0,23.000000,5.388889,253020.115873,158.800000,265000.0


In [ ]:
test_combined_median.to_excel('test_bur_demo_mean_updated_string_manipulation.xlsx')

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/LTFS_Top_up/train_bur_demo_median_updated_string_manipulation.xlsx')
df.head()

,Unnamed: 0,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,EMI,ManufacturerID,SupplierID,AGE,MonthlyIncome,State,ZiPCODE,top_up_class,Frequency_Half Yearly,Frequency_Monthly,Frequency_Quatrly,InstlmentMode_Arrear,LoanStatus_Closed,SEX_M,ID,DISBURSED-AMT/HIGH CREDIT_Median,INSTALLMENT-AMT_Median,CURRENT-BAL_Median,OVERDUE-AMT_Median,WRITE-OFF-AMT_Median,TENURE_Median,SELF-INDICATOR_Bureau_Traits_Total_Count,SELF-INDICATOR_Top_value,SELF-INDICATOR_Top_value_freq,SELF-INDICATOR_unique_values,MATCH-TYPE_Top_value,MATCH-TYPE_Top_value_freq,MATCH-TYPE_unique_values,ACCT-TYPE_Unique_count,ACCT-TYPE_Top_value,ACCT-TYPE_Top_value_freq,ACCT-TYPE_unique_values,CONTRIBUTOR-TYPE_Unique_count,CONTRIBUTOR-TYPE_Top_value,CONTRIBUTOR-TYPE_Top_value_freq,CONTRIBUTOR-TYPE_unique_values,OWNERSHIP-IND_Unique_count,OWNERSHIP-IND_Top_value,OWNERSHIP-IND_Top_value_freq,OWNERSHIP-IND_unique_values,ACCOUNT-STATUS_Unique_count,ACCOUNT-STATUS_Top_value,ACCOUNT-STATUS_Top_value_freq,ACCOUNT-STATUS_unique_values,DPD_Hist_Median_Median,CUR BAL-Hist_Median_Median,AMT OVERDUE - HIST_Median_Median,AMT PAID - HIST_Median_Median
0,0,0.186877,1,48,450000,275000.0,24000.0,1568,21946,49,35833.33,0.425187,464993,1,0,1,0,1,1,1,1,275000.0,0.0,0.0,0.0,0.0,0.0,9,True,5,True False,PRIMARY,9,PRIMARY,4,Auto Loan (Personal),5,"['Auto Loan (Personal)', 'Credit Card', 'Overd...",2,NBF,5,"['NAB', 'NBF']",1,Individual,9,['Individual'],3,Closed,5,"['Active', 'Closed', 'Delinquent']",0.0,125115.50,615.0,0.0
1,1,0.255776,333,47,485000,350000.0,10500.0,1062,34802,23,666.67,0.425187,466001,0,0,1,0,0,1,1,2,700000.0,0.0,52421.0,0.0,0.0,35.0,13,False,12,True False,PRIMARY,13,PRIMARY,7,Commercial Vehicle Loan,4,"['Auto Loan (Personal)', 'Business Loan Genera...",2,NBF,7,"['NBF', 'PRB']",2,Individual,9,"['Guarantor', 'Individual']",2,Active,7,"['Active', 'Closed']",0.0,334547.50,0.0,0.0
2,2,0.668451,1,68,690000,519728.0,38300.0,1060,127335,39,45257.00,0.425187,462030,6,0,0,1,1,0,1,3,44000.0,0.0,0.0,0.0,0.0,0.0,31,False,29,True False,PRIMARY,31,PRIMARY,8,Gold Loan,8,"['Commercial Vehicle Loan', 'Consumer Loan', '...",5,PRB,13,"['CCC', 'COP', 'NAB', 'NBF', 'PRB']",3,Individual,26,"['Guarantor', 'Individual', 'Joint']",3,Closed,19,"['Active', 'Closed', 'Delinquent']",0.0,30303.00,0.0,0.0
3,3,0.168888,125,48,480000,400000.0,11600.0,1060,25094,24,20833.33,0.425187,473335,1,0,1,0,0,1,1,7,315000.0,0.0,0.0,0.0,0.0,5.0,4,True,2,True False,PRIMARY,4,PRIMARY,2,Tractor Loan,3,"['Personal Loan', 'Tractor Loan']",1,NBF,4,['NBF'],1,Individual,4,['Individual'],2,Closed,2,"['Active', 'Closed']",0.0,195207.75,0.0,0.0
4,4,0.168888,152,44,619265,440000.0,15000.0,1046,21853,56,27313.67,0.269824,495442,2,0,1,0,1,1,1,8,200000.0,0.0,6989.0,0.0,0.0,42.0,7,False,6,True False,PRIMARY,7,PRIMARY,3,Tractor Loan,4,"['Commercial Vehicle Loan', 'Personal Loan', '...",3,NBF,4,"['NAB', 'NBF', 'PRB']",3,Individual,3,"['Guarantor', 'Individual', 'Joint']",4,Active,3,"['Active', 'Closed', 'Delinquent', 'SUIT FILED...",0.0,172109.00,0.0,0.0


In [ ]:
df.dtypes

Unnamed: 0                                    int64
PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
top_up_class                                  int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M       

In [ ]:
df['SELF-INDICATOR_Top_value'] = df['SELF-INDICATOR_Top_value'].astype('object')
df.dtypes

Unnamed: 0                                    int64
PaymentMode                                 float64
BranchID                                      int64
Tenure                                        int64
AssetCost                                     int64
AmountFinance                               float64
EMI                                         float64
ManufacturerID                                int64
SupplierID                                    int64
AGE                                           int64
MonthlyIncome                               float64
State                                       float64
ZiPCODE                                       int64
top_up_class                                  int64
Frequency_Half Yearly                         int64
Frequency_Monthly                             int64
Frequency_Quatrly                             int64
InstlmentMode_Arrear                          int64
LoanStatus_Closed                             int64
SEX_M       

In [ ]:
df.to_excel('train_bur_demo_median_updated_string_manipulation_f.xlsx')